# Improved Optimizer: Greedy + Fix Direction + Squeeze + Simulated Annealing

This experiment implements:
1. Greedy placement with more attempts
2. Fix direction rotation optimization
3. Squeeze/compaction moves
4. Simulated annealing for global optimization

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time
import copy

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """Generates a random angle weighted by abs(sin(2*angle)) for better corner packing."""
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def get_side_length(trees):
    """Get the side length of the bounding square for a list of trees."""
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

def get_bounds(trees):
    """Get bounding box of all trees."""
    if not trees:
        return (0, 0, 0, 0)
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return (float(Decimal(bounds[0]) / scale_factor),
            float(Decimal(bounds[1]) / scale_factor),
            float(Decimal(bounds[2]) / scale_factor),
            float(Decimal(bounds[3]) / scale_factor))

def has_collision(tree, placed_trees, tree_index=None):
    """Check if a tree collides with any placed trees."""
    if not placed_trees:
        return False
    placed_polygons = [p.polygon for p in placed_trees]
    if tree_index is None:
        tree_index = STRtree(placed_polygons)
    
    possible_indices = tree_index.query(tree.polygon)
    for i in possible_indices:
        if tree.polygon.intersects(placed_polygons[i]) and not tree.polygon.touches(placed_polygons[i]):
            return True
    return False

def has_any_overlap(trees):
    """Check if any trees overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
def initialize_trees_improved(num_trees, existing_trees=None, num_attempts=20):
    """Improved greedy placement with more attempts."""
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = [t.clone() for t in existing_trees]

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        # Generate trees with various angles
        angles_to_try = [0, 45, 90, 135, 180, 225, 270, 315]
        
        if not placed_trees:
            # First tree at origin with best angle
            best_tree = ChristmasTree(angle=45)  # 45 degrees often good for diagonal packing
            placed_trees.append(best_tree)
            num_to_add -= 1

        for _ in range(num_to_add):
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            best_angle = 0
            min_radius = Decimal('Infinity')

            for attempt in range(num_attempts):
                # Try different tree angles
                tree_angle = random.choice(angles_to_try) + random.uniform(-10, 10)
                tree_to_place = ChristmasTree(angle=tree_angle)
                
                # Random direction angle
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py
                    best_angle = tree_angle

            final_tree = ChristmasTree(center_x=str(best_px), center_y=str(best_py), angle=str(best_angle))
            placed_trees.append(final_tree)

    side_length = get_side_length(placed_trees)
    return placed_trees, side_length

print("Improved greedy placement function defined")

Improved greedy placement function defined


In [5]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find the optimal rotation angle to minimize bounding box."""
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg

def apply_rotation(trees, angle_deg):
    """Apply a rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center / float(scale_factor)
    rotated = shifted.dot(rot_matrix.T) + rotation_center / float(scale_factor)

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [6]:
def squeeze_trees(trees, factor=0.99):
    """Scale all tree positions toward center by a factor."""
    if len(trees) <= 1:
        return [t.clone() for t in trees]
    
    # Find center
    cx = sum(float(t.center_x) for t in trees) / len(trees)
    cy = sum(float(t.center_y) for t in trees) / len(trees)
    
    squeezed = []
    for t in trees:
        new_x = cx + (float(t.center_x) - cx) * factor
        new_y = cy + (float(t.center_y) - cy) * factor
        squeezed.append(ChristmasTree(str(new_x), str(new_y), str(t.angle)))
    
    return squeezed

def compaction(trees, step=0.01, max_iterations=100):
    """Move each tree toward center until collision."""
    if len(trees) <= 1:
        return [t.clone() for t in trees], get_side_length(trees)
    
    result = [t.clone() for t in trees]
    
    # Find center
    cx = sum(float(t.center_x) for t in result) / len(result)
    cy = sum(float(t.center_y) for t in result) / len(result)
    
    for iteration in range(max_iterations):
        improved = False
        
        for i in range(len(result)):
            # Direction toward center
            dx = cx - float(result[i].center_x)
            dy = cy - float(result[i].center_y)
            dist = math.sqrt(dx*dx + dy*dy)
            
            if dist < 0.001:
                continue
            
            # Normalize
            dx /= dist
            dy /= dist
            
            # Try to move toward center
            new_x = float(result[i].center_x) + dx * step
            new_y = float(result[i].center_y) + dy * step
            
            test_tree = ChristmasTree(str(new_x), str(new_y), str(result[i].angle))
            other_trees = result[:i] + result[i+1:]
            
            if not has_collision(test_tree, other_trees):
                result[i] = test_tree
                improved = True
        
        if not improved:
            break
    
    return result, get_side_length(result)

print("Squeeze and compaction functions defined")

Squeeze and compaction functions defined


In [7]:
def local_search_improved(trees, max_iterations=50, verbose=False):
    """Improved local search with multiple move types."""
    if len(trees) <= 1:
        return trees, get_side_length(trees)
    
    best_trees = [t.clone() for t in trees]
    best_side = get_side_length(best_trees)
    
    step_sizes = [0.02, 0.01, 0.005, 0.002, 0.001]
    rotation_deltas = [10, 5, 2, 1, 0.5]
    
    for iteration in range(max_iterations):
        improved = False
        
        # Try squeeze
        for factor in [0.995, 0.99, 0.98]:
            squeezed = squeeze_trees(best_trees, factor)
            if not has_any_overlap(squeezed):
                squeezed_side = get_side_length(squeezed)
                if squeezed_side < best_side:
                    best_trees = squeezed
                    best_side = squeezed_side
                    improved = True
                    if verbose:
                        print(f"  Squeeze improved: {float(best_side):.6f}")
                    break
        
        if improved:
            continue
        
        # Try translation and rotation for each tree
        for tree_idx in range(len(best_trees)):
            original_tree = best_trees[tree_idx].clone()
            
            # Try translation moves
            for step in step_sizes:
                for dx, dy in [(step, 0), (-step, 0), (0, step), (0, -step),
                               (step, step), (step, -step), (-step, step), (-step, -step)]:
                    test_tree = ChristmasTree(
                        str(float(original_tree.center_x) + dx),
                        str(float(original_tree.center_y) + dy),
                        str(original_tree.angle)
                    )
                    
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
            
            # Try rotation moves
            for delta in rotation_deltas:
                for d_angle in [delta, -delta]:
                    test_tree = ChristmasTree(
                        str(original_tree.center_x),
                        str(original_tree.center_y),
                        str(float(original_tree.angle) + d_angle)
                    )
                    
                    other_trees = best_trees[:tree_idx] + best_trees[tree_idx+1:]
                    if not has_collision(test_tree, other_trees):
                        test_trees = best_trees[:tree_idx] + [test_tree] + best_trees[tree_idx+1:]
                        test_side = get_side_length(test_trees)
                        
                        if test_side < best_side:
                            best_trees = test_trees
                            best_side = test_side
                            improved = True
                            break
                if improved:
                    break
            
            if improved:
                break
        
        if not improved:
            break
    
    return best_trees, best_side

print("Improved local search function defined")

Improved local search function defined


In [8]:
def simulated_annealing(trees, initial_temp=1.0, cooling_rate=0.95, max_iterations=100):
    """Simulated annealing for global optimization."""
    if len(trees) <= 1:
        return trees, get_side_length(trees)
    
    current_trees = [t.clone() for t in trees]
    current_side = get_side_length(current_trees)
    
    best_trees = [t.clone() for t in current_trees]
    best_side = current_side
    
    temp = initial_temp
    
    for iteration in range(max_iterations):
        # Generate neighbor
        tree_idx = random.randint(0, len(current_trees) - 1)
        move_type = random.choice(['translate', 'rotate'])
        
        if move_type == 'translate':
            step = random.uniform(0.001, 0.02)
            angle = random.uniform(0, 2 * math.pi)
            dx = step * math.cos(angle)
            dy = step * math.sin(angle)
            new_tree = ChristmasTree(
                str(float(current_trees[tree_idx].center_x) + dx),
                str(float(current_trees[tree_idx].center_y) + dy),
                str(current_trees[tree_idx].angle)
            )
        else:
            d_angle = random.uniform(-5, 5)
            new_tree = ChristmasTree(
                str(current_trees[tree_idx].center_x),
                str(current_trees[tree_idx].center_y),
                str(float(current_trees[tree_idx].angle) + d_angle)
            )
        
        # Check collision
        other_trees = current_trees[:tree_idx] + current_trees[tree_idx+1:]
        if has_collision(new_tree, other_trees):
            continue
        
        # Calculate new side
        new_trees = current_trees[:tree_idx] + [new_tree] + current_trees[tree_idx+1:]
        new_side = get_side_length(new_trees)
        
        # Accept or reject
        delta = float(new_side - current_side)
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_trees = new_trees
            current_side = new_side
            
            if current_side < best_side:
                best_trees = [t.clone() for t in current_trees]
                best_side = current_side
        
        # Cool down
        temp *= cooling_rate
        
        if temp < 1e-6:
            break
    
    return best_trees, best_side

print("Simulated annealing function defined")

Simulated annealing function defined


In [9]:
def optimize_config(trees, verbose=False):
    """Full optimization pipeline for a single configuration."""
    current_trees = [t.clone() for t in trees]
    current_side = get_side_length(current_trees)
    
    # 1. Fix direction rotation
    optimized_side, best_angle = optimize_rotation(current_trees)
    if optimized_side < current_side:
        current_trees = apply_rotation(current_trees, best_angle)
        current_side = optimized_side
        if verbose:
            print(f"  After rotation: {float(current_side):.6f}")
    
    # 2. Compaction
    current_trees, current_side = compaction(current_trees, step=0.01, max_iterations=50)
    if verbose:
        print(f"  After compaction: {float(current_side):.6f}")
    
    # 3. Local search
    current_trees, current_side = local_search_improved(current_trees, max_iterations=30)
    if verbose:
        print(f"  After local search: {float(current_side):.6f}")
    
    # 4. Simulated annealing (light)
    if len(current_trees) > 1:
        sa_trees, sa_side = simulated_annealing(current_trees, initial_temp=0.5, cooling_rate=0.95, max_iterations=50)
        if sa_side < current_side:
            current_trees = sa_trees
            current_side = sa_side
            if verbose:
                print(f"  After SA: {float(current_side):.6f}")
    
    # 5. Final fix direction
    optimized_side, best_angle = optimize_rotation(current_trees)
    if optimized_side < current_side:
        current_trees = apply_rotation(current_trees, best_angle)
        current_side = optimized_side
    
    return current_trees, current_side

print("Full optimization pipeline defined")

Full optimization pipeline defined


In [ ]:
# Build all configurations from 1 to 200 trees
random.seed(42)
np.random.seed(42)

all_configs = {}
all_side_lengths = {}

start_time = time.time()
current_trees = None

for n in range(1, 201):
    # Greedy placement with more attempts
    trees, side = initialize_trees_improved(n, current_trees, num_attempts=15)
    
    # Full optimization
    trees, side = optimize_config(trees, verbose=False)
    
    all_configs[n] = trees
    all_side_lengths[n] = side
    current_trees = trees
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")

In [ ]:
# Calculate total score
def calculate_score(side_lengths):
    return sum(float(s)**2 / n for n, s in side_lengths.items())

total_score = calculate_score(all_side_lengths)
print(f"Total Score: {total_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {total_score - 68.922808:.6f}")

In [ ]:
# Validate all configurations
def validate_all_configs(configs):
    invalid = []
    for n, trees in configs.items():
        if has_any_overlap(trees):
            invalid.append(n)
    return invalid

invalid_configs = validate_all_configs(all_configs)
if invalid_configs:
    print(f"WARNING: {len(invalid_configs)} configurations have overlaps: {invalid_configs[:10]}...")
else:
    print("All configurations are valid (no overlaps)")

In [ ]:
# Create and save submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in all_configs[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/002_improved_optimizer/submission.csv')
print("Submission saved!")
print(f"\nFinal Score: {total_score:.6f}")